# Metrowest machine learning self-driving car
```
The project is a self-driving car.
This notebook trains the the CNN.
Refer to the .pdf for details
  
Much of this is taken from Kiras mnist example - see reference.

References:
https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
https://www.meetup.com/Natick-Artificial-Intelligence-Meetup/
https://github.com/experiencor.self-driving-toy-car/
http://vision.stanford.edu/teaching/cs231n/
https://www.tensorflow.org/
```

# Data Loading

In [1]:
from __future__ import print_function
import keras
from keras import backend as K

import sys
sys.path.append('../lib')
from metrowestcar_dataset import Dataset

num_classes = 10


# input image dimensions
img_rows, img_cols = 64, 64

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = Dataset.data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print("\nbefore:")
print(type(x_train))
print(x_train.shape)
print(type(x_train[0][0][0][0]))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print("\nafter:")
print(type(x_train))
print(x_train.shape)
print(type(x_train[0][0][0][0]))

# convert class vectors to binary class matrices
print("\nbefore")
print(type(y_train))
print(y_train.shape)
print(type(y_train[0]))

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("\nafter")
print(type(y_train))
print(y_train.shape)
print(type(y_train[0][0]))

Using TensorFlow backend.



before:
<class 'numpy.ndarray'>
(60000, 28, 28, 1)
<class 'numpy.uint8'>

after:
<class 'numpy.ndarray'>
(60000, 28, 28, 1)
<class 'numpy.float32'>

before
<class 'numpy.ndarray'>
(60000,)
<class 'numpy.uint8'>

after
<class 'numpy.ndarray'>
(60000, 10)
<class 'numpy.float32'>


## Construct the Model

In [3]:
'''Trains a CNN on the dataset.
'''

from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

# Train the model

In [4]:
batch_size = 128
#epochs = 12
epochs = 1

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 8704/60000 [===>..........................] - ETA: 7:23 - loss: 2.7218 - acc: 0.1360

KeyboardInterrupt: 

In [ ]:
# Save weights to file

In [ ]:
# Plot loss as a function of iteration number

In [ ]:
# Evaluate performance (t-test ?)